# Шкарбаненко Михаил, Б05-907

# Задача 1. Распознавания именованных сущностей на основе fasttext

Построить модель расспознавания именованных сущностей на русском языке. В качестве данных использовать выборку NERUS (NER).

* В качестве векторного представления токенов использовать fasttext модель;
* В качестве модели использовать модель LSTM;
* Архитектуру LSTM можно выбрать произвольным образом;
* Весь процесс обучения должен быть визуализирован в tensorboard (метрики качества и пример предсказания)

P.S. Выборку можно взять из [github](https://github.com/natasha/nerus).

P.S.S. Для экономинии памяти компютера предлагается воспользоваться сжатием модели fasttext с 300-мерного к 100-мерному (на колаб не хватит оперативки на сжатие до 100-мерного вектора, поэтому работайте сразу с 300-мерными в VEC формате). А также использовать выполнить переопределения модели fasttext в VEC модель (см. sem-17).

# Решение

## 1. Подготовительная часть

### 1.1 Библиотеки

In [87]:
from copy import deepcopy

import fasttext
import fasttext.util
import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
from PIL import Image
import io
import os
from urllib.request import urlopen
from skimage.segmentation import mark_boundaries
from nltk.tokenize import RegexpTokenizer
from itertools import islice

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import requests
from scipy.stats import norm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import dvc.api

from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter

from torchvision import datasets, transforms
from nerus import load_nerus
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

### 1.2 Девайс

In [88]:
device = torch.device('mps') if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'
device

'cpu'

### 1.3 Датасет

In [89]:
ft = fasttext.load_model('cc.ru.300.bin')
fasttext.util.reduce_model(ft, 100)

In [90]:
docs = load_nerus('nerus_lenta.conllu.gz')

tokens = ft.get_words()
tokens =  ['<PAD>', '<UNK>'] + tokens
token_ids = defaultdict(lambda: 1, {token: idx for idx, token in enumerate(tokens)})

tags = ['<PAD>', '<UNK>', 'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER', 'O']
tag_ids = defaultdict(lambda: 1, {tag: idx for idx, tag in enumerate(tags)})

In [91]:
class CustomDataset(Dataset):

    def __init__(self, data, size):
        super().__init__()

        self.sents_tokens, self.sents_tags = [], []
        for tokens, tags in tqdm(data, total=size):
            tokens_idxs = list(map(token_ids.__getitem__, tokens))    
            tags_idxs = list(map(tag_ids.__getitem__, tags))
            self.sents_tokens.append(tokens_idxs)
            self.sents_tags.append(tags_idxs)
                
    def __getitem__(self, idx):
        return (torch.tensor(self.sents_tokens[idx]),
                torch.tensor(self.sents_tags[idx]))
    
    def __len__(self):
        return len(self.sents_tokens)

In [92]:
def data_generator(docs):
    for doc in docs:
        for sent in doc.sents:
            yield zip(*[(t.text, t.tag) for t in sent.tokens])

In [93]:
train_data_size, test_data_size = 50000, 1000
train_data = islice(data_generator(docs), train_data_size)
test_data = islice(data_generator(docs), test_data_size)
train_dataset = CustomDataset(train_data, train_data_size)
test_dataset = CustomDataset(test_data, test_data_size)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [94]:
print(test_dataset[0])
print(train_dataset[0])

(tensor([  8898,   2244,   5937, 195560,  99130,   5687,     33,    919,  13907,
             5,   7327,      3]), tensor([8, 8, 2, 4, 7, 8, 8, 8, 8, 8, 8, 8]))
(tensor([90090,    19, 17720,  1343,  1741, 95614,  5604,     2,     5,  2469,
         4435,    98, 42685,   545,  3638, 36269,    25,  7959,     2,  1029,
         4033,   292,     3]), tensor([8, 8, 8, 8, 4, 7, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 8, 3, 6, 8]))


In [95]:
def collate_fn(batch):
    sents_tokens, sents_tags = zip(*batch)
    new_sents_tokens = pad_sequence(sents_tokens, batch_first=True, padding_value=0)
    new_sents_tags = pad_sequence(sents_tags, batch_first=True, padding_value=0)
    return new_sents_tokens, new_sents_tags

In [96]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [97]:
def get_embedding_matrix(tokens):
    embedding_matrix = []
    for token in tqdm(tokens):
        v = ft.get_word_vector(token)
        embedding_matrix.append(v)
    embedding_matrix = np.stack(embedding_matrix)
    embedding_matrix = torch.FloatTensor(embedding_matrix)
    return embedding_matrix

## 2. Модель

### 2.1 Архитектура

In [98]:
class RNNClassifier(nn.Module):

    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, embedding_matrix, output_dim, hidden_dim, 
                 num_layers, p, bidirectional=False):
        super(RNNClassifier, self).__init__()

        
        vocab_dim, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(num_embeddings=vocab_dim, embedding_dim=embedding_dim)
        self.embedding.weight.data.copy_(embedding_matrix)
        for param in self.embedding.parameters():
            param.requires_grad = False

        self.encoder = nn.LSTM(embedding_dim, hidden_dim, num_layers,
                               bidirectional=bidirectional,
                               batch_first=True, dropout=p)
        
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input):
        embedding = self.embedding(input)
        encoded, _ = self.encoder(embedding)
        return self.linear(encoded)

### 2.2 Тренер

In [99]:
class Trainer:
    def __init__(self, 
                 logdir,
                 delimeter, 
                 loaders,
                 modelcls,
                 loss,
                 lr,
                 **model_args):
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model = modelcls(**model_args).to(self.device)
        self.loss = loss
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.logger = SummaryWriter(logdir)
        self.delimeter = delimeter
        self.trainloader, self.testloader = loaders
        self.steps = 0

        
    def _calc_loss(self, batch):
        toks, tags = [el.to(self.device) for el in batch]
        
        outputs = self.model(toks)
        (outputs.shape, tags.shape)
        
        return self.loss(outputs.transpose(1, -1), tags)

    
    def train_step(self, batch):
        self.model.zero_grad()
        self.model.train()
        
        loss = self._calc_loss(batch)
    
        loss.backward()
        self.optimizer.step()
        self.steps += 1

        return loss.cpu().item()

    
    def train_epoch(self):
        epoch_loss = 0
        
        for batch in self.trainloader:
            local_loss = self.train_step(batch)
            self.logger.add_scalar('train_loss', local_loss, self.steps)

            if self.steps % self.delimeter == 0:
                test_loss = self.test_epoch()
                self.logger.add_scalar('test_loss', test_loss, self.steps)

            epoch_loss += local_loss
        
        return epoch_loss/len(self.trainloader)

    
    def test_epoch(self, verbose=False):
        self.model.eval()

        with torch.no_grad():
            avg_loss = 0

            pred = []
            true = []
            for batch in self.testloader:
                local_loss = self._calc_loss(batch)
                avg_loss += local_loss
                
                x_batch = batch[0].to(device)
                with torch.no_grad():
                    output = self.model(x_batch)

                pred.extend(torch.argmax(output, dim=-1).cpu().numpy().flatten())
                true.extend(batch[-1].cpu().numpy().flatten())

            report = classification_report(true, pred, zero_division=0, labels=range(1, len(tag_ids)))
            
            if verbose:
                print(report)
            else:
                self.logger.add_text('Report/Test', report, self.steps)
            
        return avg_loss / len(self.testloader)
    
    def train(self, n_epochs):
        for it in tqdm(range(n_epochs)):
            epoch_loss = self.train_epoch()

## 3. Эксперименты

In [100]:
model_config = {
    'embedding_matrix': get_embedding_matrix(tokens),
    'output_dim': len(tag_ids),
    'hidden_dim': 128,
    'num_layers': 3,
    'p': 0.5
}

trainer = Trainer(
    'logs', 100,
    (train_dataloader, test_dataloader),
    RNNClassifier, torch.nn.CrossEntropyLoss(ignore_index=0),
    lr=1e-3, **model_config
)

  0%|          | 0/2000002 [00:00<?, ?it/s]

Качество модели до обучения

In [101]:
trainer.test_epoch(verbose=True)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00     447.0
           3       0.00      0.00      0.00     386.0
           4       0.00      0.00      0.00     347.0
           5       0.00      0.00      0.00      55.0
           6       0.00      0.00      0.00     289.0
           7       0.00      0.00      0.00     215.0
           8       0.00      0.00      0.00   15797.0

   micro avg       0.00      0.00      0.00   17536.0
   macro avg       0.00      0.00      0.00   17536.0
weighted avg       0.00      0.00      0.00   17536.0



tensor(2.1861)

Качество модели после обучения

In [102]:
trainer.train(5)

  0%|          | 0/5 [00:00<?, ?it/s]

In [103]:
trainer.test_epoch(verbose=True)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           2       0.86      0.96      0.91       447
           3       0.52      0.83      0.64       386
           4       0.93      0.86      0.89       347
           5       0.81      0.87      0.84        55
           6       0.44      0.78      0.56       289
           7       0.86      0.96      0.91       215
           8       0.39      0.99      0.56     15797

   micro avg       0.40      0.98      0.57     17536
   macro avg       0.60      0.78      0.66     17536
weighted avg       0.42      0.98      0.58     17536



tensor(0.0563)

# Итоги

Была построена модель для распознавания именнованных сущностей в тексте на основе LSTM. В качестве эмбеддинг-энкодера использовался предобученный fasttext. Параметры модели:

* Размерность эмбеддинг матрицы: 2 000 000 x 100
* Количество классов: 7
* Размерность скрытого вектора LSTM: 128
* Количество слоев LSTM: 3
* Вероятность droput: 0.5

Модель показала довольно неплохой перфоманс, несмотря на дизбаланс классов: $\{preision, recall, f1-score\}_{weighted} = \{42\%, 98\%, 58\%\}$.

